<a href="https://colab.research.google.com/github/dmlcks123/minkim1423.github.io/blob/main/ch3_labs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# EDA(탐색적 자료분석)와 linear regression

In [ ]:
import seaborn as sns # plot
import matplotlib.pyplot as plt # plot
import statsmodels.api as sm # 가설검정
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
# 실습1: 와인 품질 예측

# 레드와인 1599, 화이트와인 4898 append
# X_feature = 12
# Y = 와인품질평가점수

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
url2 = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"

In [ ]:
import pandas as pd

redwine = pd.read_csv(url, sep = ";", header = 0)

redwine["type"] = "red" # type 변수 추가, 레드 와인 식별
redwine.head()

In [ ]:
whitewine = pd.read_csv(url2, sep = ";", header = 0 )

whitewine["type"] = "white"
whitewine.head()

In [ ]:
# append

wine = redwine.append(whitewine)
wine.shape

In [ ]:
wine.columns = wine.columns.str.replace(" ", "_") # 변수명 공백 처리 =
wine.head()

In [ ]:
wine.describe() #요약통계량

In [ ]:
wine.quality.describe() #Series 객체의 요약통계량(질적변수)

In [ ]:
sorted(wine.quality.unique()) # quality: 1~10점인데 3~9점으로 배포되어 있음

In [ ]:
wine.quality.value_counts() # value count

In [ ]:
wine.groupby("type")["quality"].describe() #type으로 grouping 이후 quality 요약통계량

In [ ]:
wine.groupby("type")["quality"].quantile([0,0.25,0.5,0.75,1]).unstack("type")
# quality별 사분위수 파악
# unstack --> type변수 index --> coulumn으로 , dataframe 형식으로 변환

In [ ]:
# histogram
# 종류별 품질 분포

red_q = wine.loc[wine["type"] == "red", "quality"] 
white_q = wine.loc[wine["type"] == "white", "quality"]

sns.set_style("darkgrid") 
# dist 분포 plot, kenerl distibution(kde)
sns.distplot(red_q, norm_hist = True, kde = False, color = "red", label = "Red Wine")
sns.distplot(white_q, norm_hist = True, kde = False, color = "blue", label = "White Wine")
plt.title("Distribution of Quality of Wine Type")
plt.xlabel("Quality Score")
plt.ylabel("Density")
plt.legend();

# qulity 분포 5~7 점에 집중되어 있어

In [ ]:
# 화이트와인과 레드와인의 품질 차이 있을까 
wine.groupby("type")["quality"].aggregate(["std", "mean"]) #표준편차와 평균 확인하기

In [ ]:
t_stat, p_value, df = sm.stats.ttest_ind(red_q, white_q) #t값, pvalue, 자유도 반환되어 나옴
"t-stat: {:.3f}, p-value: {:.4f}".format(t_stat, p_value)

# scipy.stats / statsmodels.api # 통계적 접근시 사용하는 모듈들
# p값이 0임, 화이트와인과 레드와인의 품질 차이는 유의수준 1% 이내 통계적으로 유의함

In [ ]:
# 와인품질모델 예측
# 예측능력 평가지표 : RMSE(root mean square error), R_square

In [ ]:
# model class 
from sklearn.linear_model import LinearRegression
lr = LinearRegression(fit_intercept = True) # 상수항도 추정하겠음

In [ ]:
# 특징행렬과 대상벡터
X = wine.drop(["type", "quality"], axis = 1)
X.head()


In [ ]:
y = wine.quality
y.head()

In [ ]:
# test set, train set 분류
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
# 랜덤추출 seed값 고정(필수 아님)
# test size default : 0.25

In [ ]:
lr.fit(X_train, y_train) # train set, model에 적합합

In [ ]:
lr.coef_ # beta값

In [ ]:
X_train.columns #변수명

In [ ]:
lr.intercept_ #상수항

In [ ]:
y_pred = lr.predict(X_test) # test set 예측

In [ ]:
y_train_pred = lr.predict(X_train) # train set 에측

In [ ]:
sns.distplot(y_pred, hist = False, kde = True, color = "red", label = "y_pred")
sns.distplot(y_test, hist = False, kde = True, color = "blue", label = "y_test")
plt.xlabel("Quality Score")
plt.ylabel("Density")
plt.legend();

In [ ]:
sns.distplot(y_train_pred, hist = False, kde = True, color = "red", label = "y_train_pred")
sns.distplot(y_train, hist = False, kde = True, color = "blue", label = "y_train")
plt.xlabel("Quality Score")
plt.ylabel("Density")
plt.legend();

In [ ]:
# r_square
print("Train data\'s Accuracy : ", format(lr.score(X_train, y_train)))
print("Test data\'s Accuracy : ", format(lr.score(X_test, y_test)))

In [ ]:
# MSE
print("Train MSE\'s is ",np.round(np.sqrt(mean_squared_error(y_train, y_train_pred)),2))
print("Test MSE\'s is ",np.round(np.sqrt(mean_squared_error(y_test, y_pred)),2))

In [ ]:
# 각 변수의 p-value 확인
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train, X_train).fit()
model.summary()

In [ ]:
## 전진 선택법
variables = X_train.columns.tolist() ## 설명 변수 리스트
 
y = y_train ## 반응 변수
selected_variables = [] ## 선택된 변수들
sl_enter = 0.05 # p-value, 유의수준 0.05 이하면 선택안함#####
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
    ## 선형 모형을 적합한다.
    for col in remainder: 
        X = X_train[selected_variables+[col]]
        X = sm.add_constant(X)
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(X_train[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [ ]:
selected_variables

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.set_facecolor('white')
 
font_size = 15
plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12)
plt.plot(steps,adjusted_r_squared, marker='o')
    
plt.ylabel('Adjusted R Squared',fontsize=font_size)
plt.grid(True)
plt.show()

In [ ]:
## 후진 소거법
variables = X_train.columns.tolist() ## 설명 변수 리스트
 
y = y_train ## 반응 변수
selected_variables = variables ## 초기에는 모든 변수가 선택된 상태
sl_remove = 0.05 # p-value 값이 유의수준 5% 보다 크다면 변수제거
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(selected_variables) > 0:
    X = sm.add_constant(X_train[selected_variables])
    p_vals = sm.OLS(y,X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
    max_pval = p_vals.max() ## 최대 p-value
    if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
        remove_variable = p_vals.idxmax()
        selected_variables.remove(remove_variable)
 
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(X_train[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [ ]:
selected_variables

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.set_facecolor('white')
 
font_size = 15
plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12)
plt.plot(steps,adjusted_r_squared, marker='o')
    
plt.ylabel('Adjusted R Squared',fontsize=font_size)
plt.grid(True)
plt.show()

In [ ]:
## 전진 단계별 선택법: 유의수준보다 작으면 선택 이후 모델적합
## 모델 적합후 변수 중 가장큰 p-value 유의수준보다 크면 삭제

variables = X_train.columns.tolist() ## 설명 변수 리스트
 
y = y_train ## 반응 변수
selected_variables = [] ## 선택된 변수들
sl_enter = 0.05 # 선택기준 유의수준
sl_remove = 0.05 # 제거기준 유의수준
 
sv_per_step = [] ## 각 스텝별로 선택된 변수들
adjusted_r_squared = [] ## 각 스텝별 수정된 결정계수
steps = [] ## 스텝
step = 0
while len(variables) > 0:
    remainder = list(set(variables) - set(selected_variables))
    pval = pd.Series(index=remainder) ## 변수의 p-value
    ## 기존에 포함된 변수와 새로운 변수 하나씩 돌아가면서 
    ## 선형 모형을 적합한다.
    for col in remainder: 
        X = X_train[selected_variables+[col]]
        X = sm.add_constant(X)
        model = sm.OLS(y,X).fit()
        pval[col] = model.pvalues[col]
 
    min_pval = pval.min()
    if min_pval < sl_enter: ## 최소 p-value 값이 기준 값보다 작으면 포함
        selected_variables.append(pval.idxmin())
        ## 선택된 변수들에대해서
        ## 어떤 변수를 제거할지 고른다.
        while len(selected_variables) > 0:
            selected_X = X_train[selected_variables]
            selected_X = sm.add_constant(selected_X)
            selected_pval = sm.OLS(y,selected_X).fit().pvalues[1:] ## 절편항의 p-value는 뺀다
            max_pval = selected_pval.max()
            if max_pval >= sl_remove: ## 최대 p-value값이 기준값보다 크거나 같으면 제외
                remove_variable = selected_pval.idxmax()
                selected_variables.remove(remove_variable)
            else:
                break
        
        step += 1
        steps.append(step)
        adj_r_squared = sm.OLS(y,sm.add_constant(X_train[selected_variables])).fit().rsquared_adj
        adjusted_r_squared.append(adj_r_squared)
        sv_per_step.append(selected_variables.copy())
    else:
        break

In [ ]:
selected_variables

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.set_facecolor('white')
 
font_size = 15
plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=12)
plt.plot(steps,adjusted_r_squared, marker='o')
    
plt.ylabel('Adjusted R Squared',fontsize=font_size)
plt.grid(True)
plt.show()

In [ ]:
model.summary()
# 전진선택, 후진선택 출처:https://zephyrus1111.tistory.com/65